In [1]:
import pickle as pkl
import pandas as pd
import os
import numpy as np
from collections import Counter
import json

with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load both dataset and the set difference of previous selected works, then 

In [2]:
# os.chdir('..')

In [3]:
local_data = U.load_file('data_vLocal.pkl', 'pkl', config['DATADIR'])
# final_data = U.load_file('data_vFinal.pkl', 'pkl', config['DATADIR'])
ff_data = U.load_file('data_vFF.pkl', 'pkl', config['DATADIR'])
metadata = U.load_file('metadata.csv', 'csv', config['DATADIR'])

In [4]:
local_df = pd.DataFrame(local_data)
# final_df = pd.DataFrame(final_data)
ff_df = pd.DataFrame(ff_data)

In [5]:
columns = ff_df.columns

In [6]:
local_df = local_df.rename(columns={
    'title_y' : 'title',
    'authoryearofbirth_x':'authoryearofbirth',
    'authoryearofdeath_x':'authoryearofdeath',
    'downloads_x':'downloads',
    'subjects_x':'subjects',
    'Sub_A_x':'topic',
    'Sub_A_y':'Sub_A',
    'Sub_B_y':'Sub_B',
    'Sub_C_y':'Sub_C'
})

In [7]:
ff_df.columns

Index(['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines', 'authoryearofbirth', 'authoryearofdeath',
       'downloads', 'subjects', 'topic', 'Sub_A', 'Sub_B', 'Sub_C',
       'str_text_lines', 'passage_key'],
      dtype='object')

In [8]:
seed_works = U.load_file('seed_worksF.csv', 'csv', config['DATADIR'])

In [9]:
seed_works[seed_works['id'] == 'PG53940']

,Unnamed: 0,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,subject_set,Sub_A,Sub_B,Sub_C
7975,53938,PG53940,"Haw-Ho-Noo; Or, Records of a Tourist","Lanman, Charles",1819.0,1895.0,['en'],3.0,"{'Fishing -- North America', 'United States --...",Text,"['Description and travel', 'North America', 'F...",Description and travel,North America,Fishing


In [10]:
for_merge = seed_works[['id', 'authoryearofbirth', 'authoryearofdeath', 'downloads', 'subject_set', 'Sub_A', 'Sub_B', 'Sub_C']]

In [11]:
ff_df_null = ff_df[ff_df.authoryearofbirth.isna()]
ff_df = ff_df[~ff_df.authoryearofbirth.isna()]

ff_df_null = ff_df_null[['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines']]
null_values = ff_df_null.merge(for_merge, left_on='book_id', right_on = 'id')
null_values['topic'] = null_values['Sub_A']

In [12]:
ff_df = pd.concat((ff_df, ff_df_null))

Unique works for each of the three datasets:

In [13]:

len(local_df.book_id.unique())
# len(final_df.book_id.unique())
len(ff_df.book_id.unique())


6424

Because the final_df has a lot of text from non-English languages, I have decided to proceed as follows:
* Our overall dataset will be the `ff_df` object, corresponding to the `data_vFF.pkl` file. The filtering for this file was the most stringent.
* We will use the works not included in `ff_df`, but in `local_df`, as the training corpus for our word embeddings

In [14]:
local_df['str_text_lines'] = local_df['text_lines'].apply(str)
local_df['passage_key'] = local_df['gutenbergbookid'] + '_' + local_df['str_text_lines']

ff_df['str_text_lines'] = ff_df['text_lines'].apply(str)
ff_df['passage_key'] = ff_df['gutenbergbookid'] + '_' + ff_df['str_text_lines']


In [15]:
dataset_passages = list(ff_df.passage_key.unique())

In [16]:
ldf_index = local_df.set_index('passage_key').index
ff_index = ff_df.set_index('passage_key').index
corpus_index = ldf_index.difference(ff_index)

In [18]:
local_df_idx = local_df.set_index('passage_key')
corpus_df = local_df_idx.loc[corpus_index].reset_index()

corpus_df = corpus_df.sample(frac=.3)

In [19]:
corpus_df['joined_text'] = corpus_df.text.apply(' '.join)

In [20]:
corpus_text = " ".join(corpus_df.joined_text)

In [21]:
with open('corpus_text.txt', 'w') as f:
    f.write(corpus_text)

In [ ]:
os.chdir(config['REPODIR'])

In [ ]:
ff_out = ff_df.to_dict(orient='records')
U.write_file(ff_out, 'data_vFF.pkl', 'pkl', config['DATADIR'])

TypeError: file must have a 'write' attribute